# 🧠 Análisis Avanzado de Ventas con Pandas

Este notebook incluye técnicas más avanzadas de análisis de datos, como detección de outliers, uso de `apply()` y funciones `lambda`, y análisis multivariado sobre el archivo de ventas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar archivo CSV
df = pd.read_csv('ventas_prueba_pandas.csv', parse_dates=['Fecha'])

## 🔍 1. Detección de outliers con regla de 1.5 * IQR

In [ ]:
# Calcular IQR para Total
Q1 = df['Total'].quantile(0.25)
Q3 = df['Total'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR
limite_inferior = Q1 - 1.5 * IQR

# Filtrar outliers
outliers = df[(df['Total'] > limite_superior) | (df['Total'] < limite_inferior)]
print(f'Total de outliers detectados: {len(outliers)}')
outliers.head()

## 🛠️ 2. Crear nuevas variables con `apply()` y `lambda`

In [ ]:
# Categorizar monto total
df['Segmento'] = df['Total'].apply(lambda x: 'Bajo' if x < 500 else ('Medio' if x < 1000 else 'Alto'))
df['Dia'] = df['Fecha'].dt.day_name(locale='es')
df[['Total', 'Segmento', 'Dia']].head()

## 📊 3. Análisis multivariado: producto vs ciudad vs segmento

In [ ]:
pivot = pd.pivot_table(df, values='Total', index='Producto', columns='Ciudad', aggfunc='sum')
pivot.plot(kind='bar', figsize=(10,6), title='Total de Ventas por Producto y Ciudad')
plt.ylabel('USD')
plt.grid(True)
plt.tight_layout()
plt.show()

## 🧮 4. Ventas por día de la semana y segmento

In [ ]:
resumen_segmento = df.groupby(['Dia', 'Segmento'])['Total'].sum().unstack().fillna(0)
resumen_segmento = resumen_segmento.reindex(['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo'])
resumen_segmento.plot(kind='bar', stacked=True, figsize=(10,6), title='Ventas por Día y Segmento')
plt.ylabel('USD')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()